In [67]:
import urllib.request, json 
import requests
from bs4 import BeautifulSoup,SoupStrainer
import pkg_resources
import os
import re
import multiprocessing
import contextlib
from dateutil import parser
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from multiprocessing import Pool
from pandarallel import pandarallel

In [68]:
pandarallel.initialize(progress_bar = True)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [95]:
def getDir(dir):
    with open('data/pip_json/'+dir, 'r') as f:
        json_file = json.load(f)
        url_val, license_val = None, None
        # try adding homepage directory
        try:
            all_urls = json_file['info']['project_urls']
            all_urls = list(set(["/".join(ele.split("/")[:5]) for ele in all_urls.values() if ele.startswith('https://github.com/')]))
            print(all_urls)
            if len(all_urls)>1:
                url_val = "Unavailable"
            else:
                url_val = all_urls[0]
        except:
            url_val = "Unavailable"
    
        # try adding license
        try:
            license_val = " | ".join([ele for ele in json_file['info']['classifiers'] if 'License' in ele])
        except:
            license_val = "Unavailable"
    return url_val, license_val

In [73]:
dirs = os.listdir('data/pip_json')

In [89]:
dir_license_vals = pd.Series(dirs).parallel_apply(getDir)

In [100]:
df_linked = pd.concat([ pd.Series(dirs), pd.DataFrame(dir_license_vals)], axis = 1)
df_linked.columns =  ['package', 'data']
df_linked['package'] = df_linked['package'].apply(lambda x: x.replace('.json', ''))
df_linked['homepage'] = df_linked['data'].apply(lambda x: x[0])
df_linked['license'] = df_linked['data'].apply(lambda x: x[1])

In [104]:
df_linked['license'] = df_linked['license'].replace('', 'Unavailable')
df_linked['github repo'] = df_linked['homepage'].apply(lambda x: 'Unavailable' if 'https://github.com/' not in x else x.replace('https://github.com/',''))
df_linked['github repo'] = df_linked['github repo'].apply(lambda x: "Unavailable" if "/" not in x else x)

In [106]:
print("Total of python packages with github repository:",df_linked[df_linked['github repo']!='Unavailable'].shape[0])
df_linked['license'].value_counts()[:5]

Total of python packages with github repository: 8829


license
License :: OSI Approved :: MIT License                              3835
Unavailable                                                         2725
License :: OSI Approved :: BSD License                              2130
License :: OSI Approved :: Apache Software License                  1825
License :: OSI Approved :: GNU General Public License v3 (GPLv3)     284
Name: count, dtype: int64

In [109]:
df_linked[df_linked['github repo'] != 'Unavailable'][['package', 'github repo']].to_csv('data/inputs/package_repos.csv')

In [128]:
python_opensource = pd.read_table('data/inputs/README-Python.md', sep="|", header=0, index_col=0, skipinitialspace=True, on_bad_lines='warn').dropna(axis=1, how='all').iloc[1:]   
python_opensource['repo_name'] = python_opensource['NAME/PLACE'].apply(lambda x: "/".join(x.split("(")[1].replace(")","").split("/")[-3:-1]))

Skipping line 145: expected 8 fields, saw 10
Skipping line 441: expected 8 fields, saw 11
Skipping line 516: expected 8 fields, saw 9
Skipping line 630: expected 8 fields, saw 10
Skipping line 974: expected 8 fields, saw 9



In [132]:
python_opensource[python_opensource['repo_name'].isin(newrepos)]

,STARS,FORKS,ISSUES,LAST COMMIT,NAME/PLACE,DESCRIPTION,repo_name
NaN,53621,22054,1097,10 hours ago,[ansible](https://github.com/ansible/ansible)/4,Ansible is a radically simple IT automation pl...,ansible/ansible
NaN,34364,14600,3512,2 hours ago,[pandas](https://github.com/pandas-dev/pandas)/7,Flexible and powerful data analysis / manipula...,pandas-dev/pandas
NaN,23871,4889,56,5 days ago,[sqlmap](https://github.com/sqlmapproject/sqlm...,Automatic SQL injection and database takeover ...,sqlmapproject/sqlmap
NaN,20802,7052,2197,2 hours ago,[numpy](https://github.com/numpy/numpy)/19,The fundamental package for scientific computi...,numpy/numpy
NaN,19160,2471,21,4 hours ago,[locust](https://github.com/locustio/locust)/22,Scalable user load testing tool written in Pyt...,locustio/locust
...,...,...,...,...,...,...,...
NaN,2827,434,5,15 days ago,[praw](https://github.com/praw-dev/praw)/925,"PRAW, an acronym for ""Python Reddit API Wrappe...",praw-dev/praw
NaN,2825,463,178,8 days ago,[shapely](https://github.com/shapely/shapely)/...,Manipulation and analysis of geometric objects,shapely/shapely
NaN,2792,352,128,5 days ago,[datashader](https://github.com/holoviz/datash...,Quickly and accurately render even the largest...,holoviz/datashader
NaN,2787,222,45,"1 year, 1 month ago",[cerberus](https://github.com/pyeve/cerberus)/...,"Lightweight, extensible data validation librar...",pyeve/cerberus


In [134]:
len(newrepos)

1774

In [131]:
t = df_linked[df_linked['github repo'] != 'Unavailable'][['package', 'github repo']]
newrepos = t[~t['github repo'].isin(pd.read_csv('data/merged_data/imputed_ranks/repo_login_id.csv', index_col = 0)['repo_name'].tolist())]['github repo'].tolist()
#.to_csv('newrepos.csv')